In [39]:
# import relevant libraries
import boto3
import torch
import sagemaker
import json
import numpy as np
import PIL
import matplotlib.pyplot as plt
import torchvision
from PIL import Image
from torch import nn
from torch import optim
from torch.autograd import Variable
from torchvision import datasets, transforms

In [40]:
# display version of PyTorch
print(torch.__version__)

2.0.0


In [41]:
import sagemaker

# Start SageMaker session
sagemaker_session = sagemaker.Session()

# Specify S3 bucket and prefix for training and model data
bucket = sagemaker_session.default_bucket()
prefix = 'sagemaker/car-model'

# Specify the IAM role ARN used to access the training and model data
sagemaker_role_arn = 'arn:aws:iam::228535841440:role/Cloud_Computing'

# Set the IAM role for SageMaker
sagemaker_role = sagemaker_role_arn

# Use the sagemaker_role in your code as needed

### Getting the data

In [46]:
import os

# Specify the directory path
dir_path = os.path.expanduser("~/Desktop/tmp/cars/train")

# Create the directory if it doesn't exist
if not os.path.exists(dir_path):
    os.makedirs(dir_path)

import boto3
import os

# Specify the S3 bucket name and folder path
s3_bucket_name = 'cars-classifiers'
s3_folder_path = 'dataset/archive/cars_train/cars_train/'

# Specify the local destination path
local_path = os.path.expanduser('~/Desktop/tmp/cars/train')

# Create an S3 client
s3_client = boto3.client('s3')

# List objects in the S3 folder using list_objects_v2
paginator = s3_client.get_paginator('list_objects_v2')
operation_parameters = {'Bucket': s3_bucket_name, 'Prefix': s3_folder_path}
page_iterator = paginator.paginate(**operation_parameters)

# Download each object to the local path
for page in page_iterator:
    if 'Contents' in page:
        for s3_object in page['Contents']:
            # Extract the object key and construct the local file path
            s3_object_key = s3_object['Key']
            local_file_path = os.path.join(local_path, os.path.basename(s3_object_key))

            # Download the object to the local file path
            s3_client.download_file(s3_bucket_name, s3_object_key, local_file_path)

            print(f'Downloaded: {s3_object_key} -> {local_file_path}')

#unzip training datasets
#!unzip -o /tmp/cars/cars_train.zip

Downloaded: dataset/archive/cars_train/cars_train/00001.jpg -> /Users/folaranmiadeyeri/Desktop/tmp/cars/train/00001.jpg
Downloaded: dataset/archive/cars_train/cars_train/00002.jpg -> /Users/folaranmiadeyeri/Desktop/tmp/cars/train/00002.jpg
Downloaded: dataset/archive/cars_train/cars_train/00003.jpg -> /Users/folaranmiadeyeri/Desktop/tmp/cars/train/00003.jpg
Downloaded: dataset/archive/cars_train/cars_train/00004.jpg -> /Users/folaranmiadeyeri/Desktop/tmp/cars/train/00004.jpg
Downloaded: dataset/archive/cars_train/cars_train/00005.jpg -> /Users/folaranmiadeyeri/Desktop/tmp/cars/train/00005.jpg
Downloaded: dataset/archive/cars_train/cars_train/00006.jpg -> /Users/folaranmiadeyeri/Desktop/tmp/cars/train/00006.jpg
Downloaded: dataset/archive/cars_train/cars_train/00007.jpg -> /Users/folaranmiadeyeri/Desktop/tmp/cars/train/00007.jpg
Downloaded: dataset/archive/cars_train/cars_train/00008.jpg -> /Users/folaranmiadeyeri/Desktop/tmp/cars/train/00008.jpg
Downloaded: dataset/archive/cars_train/c

Downloaded: dataset/archive/cars_train/cars_train/00071.jpg -> /Users/folaranmiadeyeri/Desktop/tmp/cars/train/00071.jpg
Downloaded: dataset/archive/cars_train/cars_train/00072.jpg -> /Users/folaranmiadeyeri/Desktop/tmp/cars/train/00072.jpg
Downloaded: dataset/archive/cars_train/cars_train/00073.jpg -> /Users/folaranmiadeyeri/Desktop/tmp/cars/train/00073.jpg
Downloaded: dataset/archive/cars_train/cars_train/00074.jpg -> /Users/folaranmiadeyeri/Desktop/tmp/cars/train/00074.jpg
Downloaded: dataset/archive/cars_train/cars_train/00075.jpg -> /Users/folaranmiadeyeri/Desktop/tmp/cars/train/00075.jpg
Downloaded: dataset/archive/cars_train/cars_train/00076.jpg -> /Users/folaranmiadeyeri/Desktop/tmp/cars/train/00076.jpg
Downloaded: dataset/archive/cars_train/cars_train/00077.jpg -> /Users/folaranmiadeyeri/Desktop/tmp/cars/train/00077.jpg
Downloaded: dataset/archive/cars_train/cars_train/00078.jpg -> /Users/folaranmiadeyeri/Desktop/tmp/cars/train/00078.jpg
Downloaded: dataset/archive/cars_train/c

Downloaded: dataset/archive/cars_train/cars_train/00141.jpg -> /Users/folaranmiadeyeri/Desktop/tmp/cars/train/00141.jpg
Downloaded: dataset/archive/cars_train/cars_train/00142.jpg -> /Users/folaranmiadeyeri/Desktop/tmp/cars/train/00142.jpg
Downloaded: dataset/archive/cars_train/cars_train/00143.jpg -> /Users/folaranmiadeyeri/Desktop/tmp/cars/train/00143.jpg
Downloaded: dataset/archive/cars_train/cars_train/00144.jpg -> /Users/folaranmiadeyeri/Desktop/tmp/cars/train/00144.jpg
Downloaded: dataset/archive/cars_train/cars_train/00145.jpg -> /Users/folaranmiadeyeri/Desktop/tmp/cars/train/00145.jpg
Downloaded: dataset/archive/cars_train/cars_train/00146.jpg -> /Users/folaranmiadeyeri/Desktop/tmp/cars/train/00146.jpg
Downloaded: dataset/archive/cars_train/cars_train/00147.jpg -> /Users/folaranmiadeyeri/Desktop/tmp/cars/train/00147.jpg
Downloaded: dataset/archive/cars_train/cars_train/00148.jpg -> /Users/folaranmiadeyeri/Desktop/tmp/cars/train/00148.jpg
Downloaded: dataset/archive/cars_train/c

Downloaded: dataset/archive/cars_train/cars_train/00210.jpg -> /Users/folaranmiadeyeri/Desktop/tmp/cars/train/00210.jpg
Downloaded: dataset/archive/cars_train/cars_train/00211.jpg -> /Users/folaranmiadeyeri/Desktop/tmp/cars/train/00211.jpg
Downloaded: dataset/archive/cars_train/cars_train/00212.jpg -> /Users/folaranmiadeyeri/Desktop/tmp/cars/train/00212.jpg
Downloaded: dataset/archive/cars_train/cars_train/00213.jpg -> /Users/folaranmiadeyeri/Desktop/tmp/cars/train/00213.jpg
Downloaded: dataset/archive/cars_train/cars_train/00214.jpg -> /Users/folaranmiadeyeri/Desktop/tmp/cars/train/00214.jpg
Downloaded: dataset/archive/cars_train/cars_train/00215.jpg -> /Users/folaranmiadeyeri/Desktop/tmp/cars/train/00215.jpg
Downloaded: dataset/archive/cars_train/cars_train/00216.jpg -> /Users/folaranmiadeyeri/Desktop/tmp/cars/train/00216.jpg
Downloaded: dataset/archive/cars_train/cars_train/00217.jpg -> /Users/folaranmiadeyeri/Desktop/tmp/cars/train/00217.jpg
Downloaded: dataset/archive/cars_train/c

Downloaded: dataset/archive/cars_train/cars_train/00280.jpg -> /Users/folaranmiadeyeri/Desktop/tmp/cars/train/00280.jpg
Downloaded: dataset/archive/cars_train/cars_train/00281.jpg -> /Users/folaranmiadeyeri/Desktop/tmp/cars/train/00281.jpg
Downloaded: dataset/archive/cars_train/cars_train/00282.jpg -> /Users/folaranmiadeyeri/Desktop/tmp/cars/train/00282.jpg
Downloaded: dataset/archive/cars_train/cars_train/00283.jpg -> /Users/folaranmiadeyeri/Desktop/tmp/cars/train/00283.jpg
Downloaded: dataset/archive/cars_train/cars_train/00284.jpg -> /Users/folaranmiadeyeri/Desktop/tmp/cars/train/00284.jpg
Downloaded: dataset/archive/cars_train/cars_train/00285.jpg -> /Users/folaranmiadeyeri/Desktop/tmp/cars/train/00285.jpg
Downloaded: dataset/archive/cars_train/cars_train/00286.jpg -> /Users/folaranmiadeyeri/Desktop/tmp/cars/train/00286.jpg
Downloaded: dataset/archive/cars_train/cars_train/00287.jpg -> /Users/folaranmiadeyeri/Desktop/tmp/cars/train/00287.jpg
Downloaded: dataset/archive/cars_train/c

...

Downloaded: dataset/archive/cars_train/cars_train/07853.jpg -> /Users/folaranmiadeyeri/Desktop/tmp/cars/train/07853.jpg
Downloaded: dataset/archive/cars_train/cars_train/07854.jpg -> /Users/folaranmiadeyeri/Desktop/tmp/cars/train/07854.jpg
Downloaded: dataset/archive/cars_train/cars_train/07855.jpg -> /Users/folaranmiadeyeri/Desktop/tmp/cars/train/07855.jpg
Downloaded: dataset/archive/cars_train/cars_train/07856.jpg -> /Users/folaranmiadeyeri/Desktop/tmp/cars/train/07856.jpg
Downloaded: dataset/archive/cars_train/cars_train/07857.jpg -> /Users/folaranmiadeyeri/Desktop/tmp/cars/train/07857.jpg
Downloaded: dataset/archive/cars_train/cars_train/07858.jpg -> /Users/folaranmiadeyeri/Desktop/tmp/cars/train/07858.jpg
Downloaded: dataset/archive/cars_train/cars_train/07859.jpg -> /Users/folaranmiadeyeri/Desktop/tmp/cars/train/07859.jpg
Downloaded: dataset/archive/cars_train/cars_train/07860.jpg -> /Users/folaranmiadeyeri/Desktop/tmp/cars/train/07860.jpg
Downloaded: dataset/archive/cars_train/c

Downloaded: dataset/archive/cars_train/cars_train/07922.jpg -> /Users/folaranmiadeyeri/Desktop/tmp/cars/train/07922.jpg
Downloaded: dataset/archive/cars_train/cars_train/07923.jpg -> /Users/folaranmiadeyeri/Desktop/tmp/cars/train/07923.jpg
Downloaded: dataset/archive/cars_train/cars_train/07924.jpg -> /Users/folaranmiadeyeri/Desktop/tmp/cars/train/07924.jpg
Downloaded: dataset/archive/cars_train/cars_train/07925.jpg -> /Users/folaranmiadeyeri/Desktop/tmp/cars/train/07925.jpg
Downloaded: dataset/archive/cars_train/cars_train/07926.jpg -> /Users/folaranmiadeyeri/Desktop/tmp/cars/train/07926.jpg
Downloaded: dataset/archive/cars_train/cars_train/07927.jpg -> /Users/folaranmiadeyeri/Desktop/tmp/cars/train/07927.jpg
Downloaded: dataset/archive/cars_train/cars_train/07928.jpg -> /Users/folaranmiadeyeri/Desktop/tmp/cars/train/07928.jpg
Downloaded: dataset/archive/cars_train/cars_train/07929.jpg -> /Users/folaranmiadeyeri/Desktop/tmp/cars/train/07929.jpg
Downloaded: dataset/archive/cars_train/c

Downloaded: dataset/archive/cars_train/cars_train/07992.jpg -> /Users/folaranmiadeyeri/Desktop/tmp/cars/train/07992.jpg
Downloaded: dataset/archive/cars_train/cars_train/07993.jpg -> /Users/folaranmiadeyeri/Desktop/tmp/cars/train/07993.jpg
Downloaded: dataset/archive/cars_train/cars_train/07994.jpg -> /Users/folaranmiadeyeri/Desktop/tmp/cars/train/07994.jpg
Downloaded: dataset/archive/cars_train/cars_train/07995.jpg -> /Users/folaranmiadeyeri/Desktop/tmp/cars/train/07995.jpg
Downloaded: dataset/archive/cars_train/cars_train/07996.jpg -> /Users/folaranmiadeyeri/Desktop/tmp/cars/train/07996.jpg
Downloaded: dataset/archive/cars_train/cars_train/07997.jpg -> /Users/folaranmiadeyeri/Desktop/tmp/cars/train/07997.jpg
Downloaded: dataset/archive/cars_train/cars_train/07998.jpg -> /Users/folaranmiadeyeri/Desktop/tmp/cars/train/07998.jpg
Downloaded: dataset/archive/cars_train/cars_train/07999.jpg -> /Users/folaranmiadeyeri/Desktop/tmp/cars/train/07999.jpg
Downloaded: dataset/archive/cars_train/c

Downloaded: dataset/archive/cars_train/cars_train/08062.jpg -> /Users/folaranmiadeyeri/Desktop/tmp/cars/train/08062.jpg
Downloaded: dataset/archive/cars_train/cars_train/08063.jpg -> /Users/folaranmiadeyeri/Desktop/tmp/cars/train/08063.jpg
Downloaded: dataset/archive/cars_train/cars_train/08064.jpg -> /Users/folaranmiadeyeri/Desktop/tmp/cars/train/08064.jpg
Downloaded: dataset/archive/cars_train/cars_train/08065.jpg -> /Users/folaranmiadeyeri/Desktop/tmp/cars/train/08065.jpg
Downloaded: dataset/archive/cars_train/cars_train/08066.jpg -> /Users/folaranmiadeyeri/Desktop/tmp/cars/train/08066.jpg
Downloaded: dataset/archive/cars_train/cars_train/08067.jpg -> /Users/folaranmiadeyeri/Desktop/tmp/cars/train/08067.jpg
Downloaded: dataset/archive/cars_train/cars_train/08068.jpg -> /Users/folaranmiadeyeri/Desktop/tmp/cars/train/08068.jpg
Downloaded: dataset/archive/cars_train/cars_train/08069.jpg -> /Users/folaranmiadeyeri/Desktop/tmp/cars/train/08069.jpg
Downloaded: dataset/archive/cars_train/c

Downloaded: dataset/archive/cars_train/cars_train/08131.jpg -> /Users/folaranmiadeyeri/Desktop/tmp/cars/train/08131.jpg
Downloaded: dataset/archive/cars_train/cars_train/08132.jpg -> /Users/folaranmiadeyeri/Desktop/tmp/cars/train/08132.jpg
Downloaded: dataset/archive/cars_train/cars_train/08133.jpg -> /Users/folaranmiadeyeri/Desktop/tmp/cars/train/08133.jpg
Downloaded: dataset/archive/cars_train/cars_train/08134.jpg -> /Users/folaranmiadeyeri/Desktop/tmp/cars/train/08134.jpg
Downloaded: dataset/archive/cars_train/cars_train/08135.jpg -> /Users/folaranmiadeyeri/Desktop/tmp/cars/train/08135.jpg
Downloaded: dataset/archive/cars_train/cars_train/08136.jpg -> /Users/folaranmiadeyeri/Desktop/tmp/cars/train/08136.jpg
Downloaded: dataset/archive/cars_train/cars_train/08137.jpg -> /Users/folaranmiadeyeri/Desktop/tmp/cars/train/08137.jpg
Downloaded: dataset/archive/cars_train/cars_train/08138.jpg -> /Users/folaranmiadeyeri/Desktop/tmp/cars/train/08138.jpg
Downloaded: dataset/archive/cars_train/c

In [49]:
import os

# Specify the directory path
dir_path2 = os.path.expanduser("~/Desktop/tmp/cars/test")

# Create the directory if it doesn't exist
if not os.path.exists(dir_path2):
    os.makedirs(dir_path2)

import boto3
import os

# Specify the S3 bucket name and folder path
s3_bucket_name = 'cars-classifiers'
s3_folder_path2 = 'dataset/archive/cars_test/cars_test/'

# Specify the local destination path
local_path2 = os.path.expanduser('~/Desktop/tmp/cars/test')

# Create an S3 client
s3_client = boto3.client('s3')

# List objects in the S3 folder using list_objects_v2
paginator2 = s3_client.get_paginator('list_objects_v2')
operation_parameters2 = {'Bucket': s3_bucket_name, 'Prefix': s3_folder_path}
page_iterator2 = paginator.paginate(**operation_parameters)

# Download each object to the local path
for page in page_iterator:
    if 'Contents' in page:
        for s3_object in page['Contents']:
            # Extract the object key and construct the local file path
            s3_object_key2 = s3_object['Key']
            local_file_path2 = os.path.join(local_path2, os.path.basename(s3_object_key2))

            # Download the object to the local file path
            s3_client.download_file(s3_bucket_name, s3_object_key2, local_file_path2)

            print(f'Downloaded: {s3_object_key2} -> {local_file_path2}')

# unzip test datasets
#!unzip -o /tmp/cars/cars_test.zip

Downloaded: dataset/archive/cars_train/cars_train/08001.jpg -> /Users/folaranmiadeyeri/Desktop/tmp/cars/test/08001.jpg
Downloaded: dataset/archive/cars_train/cars_train/08002.jpg -> /Users/folaranmiadeyeri/Desktop/tmp/cars/test/08002.jpg
Downloaded: dataset/archive/cars_train/cars_train/08003.jpg -> /Users/folaranmiadeyeri/Desktop/tmp/cars/test/08003.jpg
Downloaded: dataset/archive/cars_train/cars_train/08004.jpg -> /Users/folaranmiadeyeri/Desktop/tmp/cars/test/08004.jpg
Downloaded: dataset/archive/cars_train/cars_train/08005.jpg -> /Users/folaranmiadeyeri/Desktop/tmp/cars/test/08005.jpg
Downloaded: dataset/archive/cars_train/cars_train/08006.jpg -> /Users/folaranmiadeyeri/Desktop/tmp/cars/test/08006.jpg
Downloaded: dataset/archive/cars_train/cars_train/08007.jpg -> /Users/folaranmiadeyeri/Desktop/tmp/cars/test/08007.jpg
Downloaded: dataset/archive/cars_train/cars_train/08008.jpg -> /Users/folaranmiadeyeri/Desktop/tmp/cars/test/08008.jpg
Downloaded: dataset/archive/cars_train/cars_trai

Downloaded: dataset/archive/cars_train/cars_train/08071.jpg -> /Users/folaranmiadeyeri/Desktop/tmp/cars/test/08071.jpg
Downloaded: dataset/archive/cars_train/cars_train/08072.jpg -> /Users/folaranmiadeyeri/Desktop/tmp/cars/test/08072.jpg
Downloaded: dataset/archive/cars_train/cars_train/08073.jpg -> /Users/folaranmiadeyeri/Desktop/tmp/cars/test/08073.jpg
Downloaded: dataset/archive/cars_train/cars_train/08074.jpg -> /Users/folaranmiadeyeri/Desktop/tmp/cars/test/08074.jpg
Downloaded: dataset/archive/cars_train/cars_train/08075.jpg -> /Users/folaranmiadeyeri/Desktop/tmp/cars/test/08075.jpg
Downloaded: dataset/archive/cars_train/cars_train/08076.jpg -> /Users/folaranmiadeyeri/Desktop/tmp/cars/test/08076.jpg
Downloaded: dataset/archive/cars_train/cars_train/08077.jpg -> /Users/folaranmiadeyeri/Desktop/tmp/cars/test/08077.jpg
Downloaded: dataset/archive/cars_train/cars_train/08078.jpg -> /Users/folaranmiadeyeri/Desktop/tmp/cars/test/08078.jpg
Downloaded: dataset/archive/cars_train/cars_trai

Downloaded: dataset/archive/cars_train/cars_train/08140.jpg -> /Users/folaranmiadeyeri/Desktop/tmp/cars/test/08140.jpg
Downloaded: dataset/archive/cars_train/cars_train/08141.jpg -> /Users/folaranmiadeyeri/Desktop/tmp/cars/test/08141.jpg
Downloaded: dataset/archive/cars_train/cars_train/08142.jpg -> /Users/folaranmiadeyeri/Desktop/tmp/cars/test/08142.jpg
Downloaded: dataset/archive/cars_train/cars_train/08143.jpg -> /Users/folaranmiadeyeri/Desktop/tmp/cars/test/08143.jpg
Downloaded: dataset/archive/cars_train/cars_train/08144.jpg -> /Users/folaranmiadeyeri/Desktop/tmp/cars/test/08144.jpg


In [66]:
import os
from PIL import Image
import torch
import torchvision.transforms as transforms
from torch.utils.data import Dataset, random_split
from sklearn.model_selection import train_test_split

class CustomCarDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_files = os.listdir(root_dir)

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, index):
        image_file = self.image_files[index]
        image_path = os.path.join(self.root_dir, image_file)
        image = Image.open(image_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        return image

# Define data transformations
train_transforms = transforms.Compose([
    transforms.Resize(255),
    transforms.RandomCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

test_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

# Load and transform the datasets with CustomCarDataset
random_seed = 42
train_dataset = CustomCarDataset('/Users/folaranmiadeyeri/Desktop/tmp/cars/train', transform=train_transforms)
# train_ratio = 0.7
# valid_ratio = 0.3

# train_size = int(train_ratio * len(dataset))
# valid_size = int(valid_ratio * len(dataset))
# assert train_size + valid_size == len(dataset), "Dataset sizes do not add up to total length of the dataset."
# train_dataset, valid_dataset = random_split(dataset, [train_size, valid_size])

# Divide the training data into training (80%) and validation (20%)
train_dataset, valid_dataset = train_test_split(train_dataset, train_size=0.7, random_state=random_seed)

test_dataset = CustomCarDataset('/Users/folaranmiadeyeri/Desktop/tmp/cars/test', transform=test_transforms)

# Create data loaders
number_of_workers = 0
batch_size = 8

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=number_of_workers)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size, shuffle=True, num_workers=number_of_workers)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True, num_workers=number_of_workers)


In [75]:
import scipy.io as sio
import json
import base64
import numpy as np

# Load .mat file
mat_data = sio.loadmat('cars_annos.mat')

# Define custom JSON encoder for NumPy arrays
class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return super(NumpyEncoder, self).default(obj)

# Convert MATLAB data to Python objects
# Note: mat_data is a dictionary-like object with keys as variable names
# and values as the loaded data
python_data = {}
for key in mat_data:
    # Check if the data is binary (bytes) type
    if isinstance(mat_data[key], bytes):
        # Encode binary data as base64
        python_data[key] = base64.b64encode(mat_data[key]).decode('utf-8')
    else:
        python_data[key] = mat_data[key]

# Convert Python objects to JSON
json_data = json.dumps(python_data, cls=NumpyEncoder)

# Write JSON to file
with open('cat_to_name.json', 'w') as json_file:
    json_file.write(json_data)

In [ ]:
# obtain label mapping
!wget -cq https://raw.githubusercontent.com/udacity/pytorch_challenge/master/cat_to_name.json

In [ ]:
# display label mapping
import json

with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)
    
cat_to_name

In [ ]:
# explore images
train_loader = torch.utils.data.DataLoader(train_datasets, batch_size=batch_size, 
                                           num_workers=number_of_workers)

# get some random training images
dataiter = iter(train_loader)
images, labels = dataiter.next()

In [ ]:
# function to show an image
def imshow(img):
    img = img / 2 + 0.456  # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))

In [ ]:
# show images
imshow(torchvision.utils.make_grid(images))

In [ ]:
# upload datasets to s3 bucket 
train_input_path   = sagemaker_session.upload_data('flower_data/train', bucket=bucket, key_prefix=prefix+'/train')
valid_input_path = sagemaker_session.upload_data('flower_data/valid', bucket=bucket, key_prefix=prefix+'/valid')
test_input_path = sagemaker_session.upload_data('test', bucket=bucket, key_prefix=prefix+'/test')

## Training

In [ ]:
# specify folder for model output
# model_output_location = 's3://{}/{}/output'.format(bucket, prefix)

from sagemaker.pytorch import PyTorch

py_estimator = PyTorch(entry_point='pytorch-cnn.py',
                    role=role,
                    framework_version='0.4.0',
                    train_instance_count=1,
                    train_instance_type='ml.p2.xlarge',
                    hyperparameters={
                        'epochs': 80,
                        'lr' : 0.01,
                        'backend': 'gloo'
                    })

In [ ]:
py_estimator.fit({'training': train_input_path, 'validation': valid_input_path})

## Inference

In [ ]:
predictor = py_estimator.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

In [ ]:
def predict(image_path, model, topk=5):
    ''' Predict the class (or classes) of an image using a trained deep learning model.
    '''
    
    # TODO: Implement the code to predict the class from an image file
    
    model.to(device)
    model.eval()
    image = process_image(image_path).unsqueeze_(0)
    output = model(image.to(device)) / 100
    output = output.float().cpu()
    return torch.topk(output, topk)

In [ ]:
def plot_probs(names, values):
    
    # Horizontal plot adapted from Matplotlib documentation
    # https://matplotlib.org/gallery/lines_bars_and_markers/barh.html
    plt.rcdefaults()
    fig, ax = plt.subplots()
    
    y_pos = np.arange(len(names))
    
    ax.barh(y_pos, values, color='blue', ecolor='black')
    ax.set_yticks(y_pos)
    ax.set_yticklabels(names)
    ax.invert_yaxis()
    
    plt.show()

In [ ]:
# TODO: Display an image along with the top 5 classes
image = (test + '/73/' + 'image_00258.jpg' )
imshow(process_image(image2).numpy())
plot_probs(flower_names, flower_probs)

<img src="pic1.JPG">

<img src="pic2.JPG">